<a href="https://colab.research.google.com/github/Robert2004-AI/LearnComputerVisionWithMithun/blob/main/ADL_Assigment01_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group No - 44

**Group Member Names:**

SAKTHI R (2023aa05940)

ROBERTSEKAR R (2023aa05823)

RAVISHANKAR R (2023aa05171

NIMBALKAR PRITEESH DADASAHEB (2023aa05950)

# Task 3:

Train an appropriate deep convolutional autoencoder with same dimension of latent space. Calculate the reconstruction error fand compare that with a single hidden layer autoencoder (with sigmoid activation at the autoencoder and linear at the decoder) for the test dataset. What will be the reconstruction error if the hidden nodes are distributed equally (approximately) among 3 hidden layers in a new 3 hidden layer autoencoder with sigmoid activation at the autoencoder and linear at the decoder final layer?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.datasets import fetch_openml
from tensorflow.keras.utils import to_categorical

# ---- Load MNIST Dataset ----
def load_data():
    """
    Load the MNIST dataset from OpenML, normalize the pixel values to [0, 1],
    and split into training and testing sets.
    """
    mnist = fetch_openml('mnist_784', version=1)  # Fetch MNIST dataset
    X = mnist.data / 255.0  # Normalize pixel values to [0, 1]
    y = mnist.target.astype(int)  # Convert target labels to integers
    # Split the data into training (70%) and testing (30%) sets
    return train_test_split(X, y, test_size=0.3, random_state=42)

X_train, X_test, y_train, y_test = load_data()


In [2]:
# Deep Convolutional Autoencoder
latent_dim = 50  # Set the latent space dimensionality

def build_autoencoder(input_dim, latent_dim):
    """
    Build a single-layer linear autoencoder model.

    Parameters:
        input_dim (int): Dimensionality of the input data.
        latent_dim (int): Dimensionality of the latent (compressed) space.

    Returns:
        autoencoder (Model): Compiled autoencoder model.
    """
    # Define the input layer with the same shape as the input data
    input_layer = layers.Input(shape=(input_dim,))

    # Encoder: Compress input to latent space of size 'latent_dim' with linear activation
    encoded = layers.Dense(latent_dim, activation='linear')(input_layer)

    # Decoder: Reconstruct input from the latent space with linear activation
    decoded = layers.Dense(input_dim, activation='linear')(encoded)

    # Create the autoencoder model combining encoder and decoder
    autoencoder = models.Model(input_layer, decoded)

    # Compile the model with Adam optimizer and Mean Squared Error (MSE) loss
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

def build_deep_autoencoder(input_shape, latent_dim):
    """
    Build a deep convolutional autoencoder model.

    Parameters:
        input_shape (tuple): Shape of the input data (e.g., (28, 28, 1) for MNIST images).
        latent_dim (int): Dimensionality of the latent (compressed) space.

    Returns:
        autoencoder (Model): Compiled autoencoder model.
    """
    # Input layer for image data with specified shape
    input_layer = layers.Input(shape=input_shape)

    # Encoder: First convolutional layer with 32 filters, 3x3 kernel, ReLU activation, and same padding
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    # Downsampling using max pooling to reduce spatial dimensions
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Encoder: Second convolutional layer with 64 filters, 3x3 kernel, ReLU activation, and same padding
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    # Further downsampling using max pooling
    x = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Flatten the feature maps into a 1D vector for the fully connected layer
    x = layers.Flatten()(x)
    # Fully connected layer to compress data into the latent space
    encoded = layers.Dense(latent_dim, activation='relu')(x)
    encoded.name = "dense_2"  # Naming the layer for easier access

    # Decoder: Expand latent representation back to original input shape
    # Fully connected layer to reconstruct the flattened input
    decoder_input = layers.Reshape((latent_dim,))(encoded)
    decoder = layers.Dense(np.prod(input_shape), activation='sigmoid')(decoder_input)
    # Reshape the flattened output back into the original input dimensions
    decoded = layers.Reshape(input_shape)(decoder)

    # Create the autoencoder model combining encoder and decoder
    autoencoder = models.Model(input_layer, decoded)

    # Compile the model with Adam optimizer and Mean Squared Error (MSE) loss
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

# ---- Main Execution ----

# Reshape training and test data into 4D format for convolutional layers
# Each image is reshaped to (28, 28, 1) to represent height, width, and channels
X_train_reshaped = X_train.values.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.values.reshape(-1, 28, 28, 1)

# Build the deep convolutional autoencoder
deep_autoencoder = build_deep_autoencoder((28, 28, 1), latent_dim)

# Train the autoencoder using the reshaped training data
# The goal is to reconstruct the input images while learning a compressed latent representation
print("Training Deep Convolutional Autoencoder...")
deep_autoencoder.fit(X_train_reshaped, X_train_reshaped, epochs=10, batch_size=256)

# Evaluate the reconstruction error on the test set
# This quantifies how well the autoencoder reconstructs unseen data
reconstruction_error_deep = deep_autoencoder.evaluate(X_test_reshaped, X_test_reshaped, verbose=0)
print(f"Reconstruction Error (Deep Autoencoder): {reconstruction_error_deep:.4f}")


# Train a single hidden layer autoencoder for comparison
print("Training Single Hidden Layer Autoencoder")
# Build a simple autoencoder with a single dense layer
autoencoder = build_autoencoder(X_train.shape[1], latent_dim)
# Train the simple autoencoder
autoencoder.fit(X_train, X_train, epochs=10, batch_size=256, shuffle=True)

# Evaluate the reconstruction error for the single-layer autoencoder
reconstruction_error_single = autoencoder.evaluate(X_test, X_test, verbose=0)
print(f"Reconstruction Error (Single Hidden Layer): {reconstruction_error_single:.4f}")

Training Deep Convolutional Autoencoder...
Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 0.1045
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0308
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0207
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0167
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0144
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0129
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0119
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0113
Epoch 9/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0108
Epoch 10/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0104
Reconstruction Error (Deep Autoencoder): 0.0103
Training Single Hidden Layer Autoencoder
Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0655
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0206
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.